# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import requests
import matplotlib.pyplot as plt
import numpy as np
import os

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/kimiri/Downloads/Starter_Code 7/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,calama,-22.4667,-22.4667,24.27,7,0,9.77,CL,1726603789
1,1,enewetak,11.3474,11.3474,28.70,72,93,2.30,MH,1726604063
2,2,puerto natales,-51.7236,-51.7236,1.25,100,100,6.69,CL,1726604065
3,3,port elizabeth,-33.9180,-33.9180,16.16,94,40,5.14,ZA,1726604066
4,4,port mathurin,-19.6833,-19.6833,21.78,82,5,7.72,MU,1726604067


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
import hvplot.pandas

# Define colors
colors = ['pink', 'red', 'black', 'purple']  # Orange, Brown, Blue, Green

# Create a scatter plot using hvplot with predefined colors
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    hover_cols=['City'],
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=500,
    frame_height=300,
    color=colors,  # Predefined colors
    cmap=None,  # Disable colormap
    colorbar=False,  # Disable colorbar
    tiles='CartoLight',
)

# Show the plot
scatter_plot

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] > 30) & (city_data_df['Max Temp'] < 80) & 
                                  (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,coxim,-18.5067,-18.5067,37.56,23,0,2.25,BR,1726603990
42,42,lakki,32.6079,32.6079,33.14,32,0,2.97,PK,1726604119
78,78,brownfield,33.1812,33.1812,31.33,38,0,8.30,US,1726604167
82,82,daud khel,32.8750,32.8750,31.82,38,0,2.71,PK,1726604172
158,158,iowa falls,42.5225,42.5225,30.08,35,0,7.20,US,1726604406


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,calama,CL,-22.4667,-22.4667,7,
1,enewetak,MH,11.3474,11.3474,72,
2,puerto natales,CL,-51.7236,-51.7236,100,
3,port elizabeth,ZA,-33.9180,-33.9180,94,
4,port mathurin,MU,-19.6833,-19.6833,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "apiKey":geoapify_key,
    "limit": 1,
    "categories": "hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    response = requests.get(base_url,params)
    
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# To display the plot (only if automatic display does not work)
hv.output(map_plot)


# Sample data to illustrate the structure
data = {
    'Lat': [37.7749, 34.0522, 40.7128],
    'Lon': [-122.4194, -118.2437, -74.0060],
    'Hotel Name': ['Hotel A', 'Hotel B', 'Hotel C'],
    'Country': ['USA', 'USA', 'USA']
}
hotel_df = pd.DataFrame(data)

# Ensure the column names are correct
if 'Lon' not in hotel_df.columns or 'Lat' not in hotel_df.columns:
    print("Error: 'Lon' and 'Lat' columns not found in the DataFrame.")
else:
    # Configure the map plot with hover information
    map_plot = hotel_df.hvplot.points(
        x='Lon',
        y='Lat',
        geo=True,
        tiles='EsriImagery',
        frame_width=800,
        frame_height=600,
        size=10,  # Adjust the size as needed
        hover_cols=['Hotel Name', 'Country']  # Add columns to display in the hover tooltip
    )

    # Save the plot as an image
    hv.extension('bokeh')
    renderer = hv.renderer('bokeh')
    renderer.save(map_plot, 'map_plot')

    print("Map plot saved as map_plot.png")

NameError: name 'hv' is not defined

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)